# Build a Semantic Search Engine in 5 Minutes

This notebook accompanies the tutorial at https://qdrant.tech/documentation/tutorials-basics/search-beginners/

# Overview

If you are new to vector search engines, this tutorial is for you. In 5 minutes you will build a semantic search engine for science fiction books. After you set it up, you will ask the engine about an impending alien threat. Your creation will recommend books as preparation for a potential space attack.

# 1. Create a Qdrant Cluster

If you do not already have a Qdrant cluster, follow these steps to create one:

- Register for a [Qdrant Cloud account](https://cloud.qdrant.io/) using your email, Google, or Github credentials.
- Under Create a Free Cluster, enter a cluster name and select your preferred cloud provider and region.
- Click Create Free Cluster.
- Copy the API key when prompted and store it somewhere safe as it won't be displayed again.
- Copy the Cluster Endpoint. It should look something like `https://xxx.cloud.qdrant.io`.


# 2. Set up a Client Connection

First, install the Qdrant Client for Python. This library allows you to interact with Qdrant from Python code.

In [ ]:
!pip install qdrant-client

Next, create a client connection to your Qdrant cluster. Ensure that you have added `QDRANT_URL` and `QDRANT_API_KEY` as secrets.

In [ ]:
from google.colab import userdata

QDRANT_URL=userdata.get("QDRANT_URL")
QDRANT_API_KEY=userdata.get("QDRANT_API_KEY")

In [ ]:
from qdrant_client import QdrantClient, models

client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
    cloud_inference=True
)

The `cloud_inference=True` parameter enables Qdrant Cloud's [inference](https://qdrant.tech/documentation/concepts/inference/) capabilities, allowing the cluster to generate vector embeddings without the need to manage your own embedding infrastructure.

# 3. Create a Collection

All data in Qdrant is organized within [collections](https://qdrant.tech/documentation/concepts/collections/). Since you're storing books, let's create a collection named `my_books`.

In [ ]:
COLLECTION_NAME="my_books"

client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=models.VectorParams(
        size=384,  # Vector size is defined by the model
        distance=models.Distance.COSINE,
    ),
)

- The `size` parameter defines the dimensionality of the vectors for the collection. 384 corresponds to the output dimensionality of the embedding model used in this tutorial.
- The `distance` parameter specifies the function used to measure the distance between two points.

# 4. Upload Data to the Cluster

The dataset consists of a list of science fiction books. Each entry has a name, author, publication year, and short description.

In [ ]:
documents = [
    {
        "name": "The Time Machine",
        "description": "A man travels through time and witnesses the evolution of humanity.",
        "author": "H.G. Wells",
        "year": 1895,
    },
    {
        "name": "Ender's Game",
        "description": "A young boy is trained to become a military leader in a war against an alien race.",
        "author": "Orson Scott Card",
        "year": 1985,
    },
    {
        "name": "Brave New World",
        "description": "A dystopian society where people are genetically engineered and conditioned to conform to a strict social hierarchy.",
        "author": "Aldous Huxley",
        "year": 1932,
    },
    {
        "name": "The Hitchhiker's Guide to the Galaxy",
        "description": "A comedic science fiction series following the misadventures of an unwitting human and his alien friend.",
        "author": "Douglas Adams",
        "year": 1979,
    },
    {
        "name": "Dune",
        "description": "A desert planet is the site of political intrigue and power struggles.",
        "author": "Frank Herbert",
        "year": 1965,
    },
    {
        "name": "Foundation",
        "description": "A mathematician develops a science to predict the future of humanity and works to save civilization from collapse.",
        "author": "Isaac Asimov",
        "year": 1951,
    },
    {
        "name": "Snow Crash",
        "description": "A futuristic world where the internet has evolved into a virtual reality metaverse.",
        "author": "Neal Stephenson",
        "year": 1992,
    },
    {
        "name": "Neuromancer",
        "description": "A hacker is hired to pull off a near-impossible hack and gets pulled into a web of intrigue.",
        "author": "William Gibson",
        "year": 1984,
    },
    {
        "name": "The War of the Worlds",
        "description": "A Martian invasion of Earth throws humanity into chaos.",
        "author": "H.G. Wells",
        "year": 1898,
    },
    {
        "name": "The Hunger Games",
        "description": "A dystopian society where teenagers are forced to fight to the death in a televised spectacle.",
        "author": "Suzanne Collins",
        "year": 2008,
    },
    {
        "name": "The Andromeda Strain",
        "description": "A deadly virus from outer space threatens to wipe out humanity.",
        "author": "Michael Crichton",
        "year": 1969,
    },
    {
        "name": "The Left Hand of Darkness",
        "description": "A human ambassador is sent to a planet where the inhabitants are genderless and can change gender at will.",
        "author": "Ursula K. Le Guin",
        "year": 1969,
    },
    {
        "name": "The Three-Body Problem",
        "description": "Humans encounter an alien civilization that lives in a dying system.",
        "author": "Liu Cixin",
        "year": 2008,
    },
]


Store each book as a [point](https://qdrant.tech/documentation/concepts/points/) in the `my_books` collection, with each point consisting of a [unique ID](https://qdrant.tech/documentation/concepts/points/#point-ids), a [vector](https://qdrant.tech/documentation/concepts/vectors/) generated from the description, and a [payload](https://qdrant.tech/documentation/concepts/payload/) containing the
book's metadata:

In [ ]:
# Define the embedding model used by Cloud Inference
EMBEDDING_MODEL="sentence-transformers/all-minilm-l6-v2"

client.upload_points(
    collection_name=COLLECTION_NAME,
    points=[
        models.PointStruct(
            id=idx,
            vector=models.Document(
                text=doc["description"],
                model=EMBEDDING_MODEL # Cloud Inference generates embeddings with this model
            ),
            payload=doc
        )
        for idx, doc in enumerate(documents)
    ],
)

This code tells Qdrant Cloud to use the `sentence-transformers/all-minilm-l6-v2` embedding model to generate vector embeddings from the book descriptions. This is one of the free models available on Qdrant Cloud. For a list of the available free and paid models, refer to the Inference tab of the Cluster Detail page in the Qdrant Cloud Console.

# 5. Query the Engine

Now that the data is stored in Qdrant, you can query it and receive semantically relevant results.

In [ ]:
hits = client.query_points(
    collection_name=COLLECTION_NAME,
    query=models.Document(
        text="alien invasion",
        model=EMBEDDING_MODEL
    ),
    limit=3,
).points

for hit in hits:
    print(hit.payload, "score:", hit.score)

This query uses the same embedding model to generate a vector for the query “alien invasion”. The search engine then looks for the three most similar vectors in the collection and returns their payloads and similarity scores.

The search engine returns the three most relevant books related to an alien invasion. Each is assigned a score indicating its similarity to the query:

## Narrow down the Query

How about the most recent book from the early 2000s? Qdrant, allows you to narrow down query results by applying a filter. To filter for books published after the year 2000, you can [filter](https://qdrant.tech/documentation/concepts/filtering/) on the `year` field in the payload.

Before filtering on a payload field, create a [payload index](https://qdrant.tech/documentation/concepts/indexing/#payload-index) for that field:

In [ ]:
client.create_payload_index(
    collection_name=COLLECTION_NAME,
    field_name="year",
    field_schema="integer",
)

In a production environment, create payload indexes before uploading data to get the maximum benefit from indexing.

Now you can apply a filter to the query:



In [ ]:
hits = client.query_points(
    collection_name=COLLECTION_NAME,
    query=models.Document(
        text="alien invasion",
        model=EMBEDDING_MODEL
    ),
    query_filter=models.Filter(
        must=[models.FieldCondition(key="year", range=models.Range(gte=2000))]
    ),
    limit=1,
).points

for hit in hits:
    print(hit.payload, "score:", hit.score)

The results have been narrowed down to one result from 2008.

# Next Steps

Congratulations, you have just created your very first search engine! Trust us, the rest of Qdrant is not that complicated, either. For your next tutorial, try building your own [hybrid search service](https://qdrant.tech/documentation/tutorials-search-engineering/hybrid-search-fastembed/) or take the free [Qdrant Essentials course](https://qdrant.tech/course/essentials/).